In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
plt.ioff();

In [ ]:
def init_fig():
    global fig
    fig = plt.figure()
    plt.close()
    

def add_scatter_xy():
    global fig
    global x,y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.scatter(x,y)
    plt.show()

def add_scatter_y():
    global fig
    global y
    plt.get_current_fig_manager().canvas.figure = fig
    x = (np.ones_like(y)*range(y.shape[-1])).T
    plt.scatter(x,y.T, s=0.1)
    plt.show()

def add_line_xy():
    global fig
    global x, y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.plot(x, y.T)
    plt.show()

def add_line_y():
    global fig
    global y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.plot(y.T)
    plt.show()

def show_fig():
    global fig
    plt.get_current_fig_manager().canvas.figure = fig
    plt.show()

def set_label_title():
    global xlabel, ylabel, title
    plt.get_current_fig_manager().canvas.figure = fig
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

def set_axis_lim():
    global fig
    global xlim_high, xlim_low
    global ylim_high, ylim_low
    plt.get_current_fig_manager().canvas.figure = fig
    plt.xlim([xlim_low, xlim_high])
    plt.ylim([ylim_low, ylim_high])
    plt.show()

def set_autoscale():
    global fig
    plt.get_current_fig_manager().canvas.figure = fig
    plt.gca().relim()
    plt.gca().autoscale()
    plt.show()

def calc_var():
    global y
    y=np.var(w, axis=0)

def calc_quadratures():
    global quadratures
    global pulseNum
    global hd_idx
    global pulse_id
    global w, t
    global y

    quadrature_method = lambda volts: volts.reshape(-1,volts.shape[-1])[:,hd_idx].reshape(volts.shape[:-1])

    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))

    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]

    quadratures = quadrature_method(volts)
    y = quadratures[:,pulse_id]

def calc_moving_var():
    global quadratures
    global pulse_id
    global y
    quadrature = quadratures[:, pulse_id]
    K = 200
    conv = np.ones(K)/K
    moving_var = np.convolve(quadrature**2, conv, mode="same") - np.convolve(quadrature, conv, mode="same")**2 
    y = 10*np.log10(moving_var)

def save_fig_pdf():
    global fig
    global comment
    plt.get_current_fig_manager().canvas.figure = fig
    plt.savefig(f'{comment}.pdf')
    plt.close()

def save_fig_png():
    global fig
    global comment
    plt.get_current_fig_manager().canvas.figure = fig
    plt.savefig(f'{comment}.png', dpi=300)
    plt.close()

def graph_quadratures():
    global fig
    global quadratures
    global pulse_id
    global hist_density_max
    fig = plt.figure(figsize=(10,3))
    plt.subplot2grid((1,3),(0,0), colspan=2, label='scatter')
    plt.subplot2grid((1,3),(0,2), label='hist')

    axes_dict = {i.get_label():i for i in plt.gcf().axes }

    plt.sca(axes_dict['scatter'])
    plt.scatter(range(len(quadratures[:,pulse_id])),quadratures[:,pulse_id], s=0.1, color='black')
    plt.xlabel('sample number')
    plt.ylabel('HD Voltage [V]')
    plt.ylim([-0.06,0.06])
    plt.xlim([0, len(quadratures[:,pulse_id])])

    plt.sca(axes_dict['hist'])
    plt.hist(quadratures[:, pulse_id], density=True,bins=100, color='black')
    plt.xlabel('HD Voltage [V]')
    plt.ylabel('Probability density')
    plt.xticks(rotation=60)
    plt.xlim([-0.06,0.06])
    plt.ylim([0,hist_density_max])
    plt.tight_layout()
    plt.show()


In [ ]:
def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_init_fig = widgets.Button(description='初期化')
    button_set_label_title = widgets.Button(description='ラベル・タイトル')
    button_set_axis_lim = widgets.Button(description='軸範囲設定')
    button_set_autoscale = widgets.Button(description='自動範囲設定')
    button_add_line_xy = widgets.Button(description='プロット(x,y)')
    button_add_line_y = widgets.Button(description='プロット(y)')
    button_add_scatter_xy = widgets.Button(description='散布図(x,y)')
    button_add_scatter_y = widgets.Button(description='散布図(y)')
    button_save_fig_pdf = widgets.Button(description='save_fig_pdf')
    button_save_fig_png = widgets.Button(description='save_fig_png')
    button_show_fig = widgets.Button(description='描写')
    filechooser = FileChooser('./')
    button_load_as_x_npy = widgets.Button(description='x.npy読込')
    button_load_as_y_npy = widgets.Button(description='y.npy読込')
    button_load_as_t_npy = widgets.Button(description='t.npy読込')
    button_load_as_w_npy = widgets.Button(description='w.npy読込')
    button_load_as_x_txt = widgets.Button(description='x.txt読込')
    button_load_as_y_txt = widgets.Button(description='y.txt読込')
    button_load_as_quadratures_txt = widgets.Button(description='quadratures.txt読込')
    button_input_field = widgets.Button(description='変数反映')

    text_comment = widgets.Text(value='',description='comment')
    text_xlabel = widgets.Text(value='',description='xlabel')
    text_ylabel = widgets.Text(value='',description='ylabel')
    text_title = widgets.Text(value='',description='title')
    text_xlim_high = widgets.Text(value='None', description='xlim_high')
    text_xlim_low = widgets.Text(value='None', description='xlim_low')
    text_ylim_high = widgets.Text(value='None', description='ylim_high')
    text_ylim_low = widgets.Text(value='None', description='ylim_low')
    int_pulseNum = widgets.IntText(value=5,description='pulseNum')
    int_hd_idx = widgets.IntText(value=330, description='hd_idx')
    int_pulse_id = widgets.IntText(value=0,description='pulse_id')
    float_hist_density_max = widgets.FloatText(value=50,description='hist_density_max', style= {'description_width': 'initial'})

    
    

    button_x_transpose = widgets.Button(description='x転置')
    button_y_transpose = widgets.Button(description='y転置')
    button_calc_var = widgets.Button(description='var計算')
    button_calc_quadratures = widgets.Button(description='quad計算')
    button_calc_moving_var = widgets.Button(description='移動分散計算')

    button_graph_quadratures = widgets.Button(description='graph_quadratures', layout={'width':'auto'})



    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_init_fig.on_click(wrapped_func_factory(init_fig))
    button_set_label_title.on_click(wrapped_func_factory(set_label_title))
    button_add_line_xy.on_click(wrapped_func_factory(add_line_xy))
    button_add_line_y.on_click(wrapped_func_factory(add_line_y))
    button_add_scatter_xy.on_click(wrapped_func_factory(add_scatter_xy))
    button_add_scatter_y.on_click(wrapped_func_factory(add_scatter_y))
    button_show_fig.on_click(wrapped_func_factory(show_fig))
    button_set_axis_lim.on_click(wrapped_func_factory(set_axis_lim))
    button_set_autoscale.on_click(wrapped_func_factory(set_autoscale))
    button_save_fig_pdf.on_click(wrapped_func_factory(save_fig_pdf))
    button_save_fig_png.on_click(wrapped_func_factory(save_fig_png))
    button_calc_var.on_click(wrapped_func_factory(calc_var))
    button_calc_quadratures.on_click(wrapped_func_factory(calc_quadratures))
    button_calc_moving_var.on_click(wrapped_func_factory(calc_moving_var))
    button_graph_quadratures.on_click(wrapped_func_factory(graph_quadratures))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global x, y
            global w, t
            if variable_name == 'x':
                x = np.load(choosed_file_path)
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = np.load(choosed_file_path)
                print(f'y.shape={y.shape}')
            elif variable_name == 'w':
                w = np.load(choosed_file_path)
                print(f'w.shape={w.shape}')
            elif variable_name == 't':
                t = np.load(choosed_file_path)
                print(f't.shape={t.shape}')
        return load_npy
    def load_txt_factory(variable_name):
        def load_txt():
            choosed_file_path = filechooser.selected
            global x, y
            global quadratures
            if variable_name == 'x':
                x = np.loadtxt(choosed_file_path)
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = np.loadtxt(choosed_file_path)
                print(f'y.shape={y.shape}')
            elif variable_name == 'quadratures':
                quadratures = np.loadtxt(choosed_file_path)
                print(f'quadratures.shape={quadratures.shape}')
        return load_txt
    def transpose_factory(variable_name):
        def transpose():
            global x,y
            if variable_name == 'x':
                x = x.T
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = y.T
                print(f'y.shape={y.shape}')
        return transpose
    button_load_as_x_npy.on_click(wrapped_func_factory(load_npy_factory('x')))
    button_load_as_y_npy.on_click(wrapped_func_factory(load_npy_factory('y')))
    button_load_as_w_npy.on_click(wrapped_func_factory(load_npy_factory('w')))
    button_load_as_t_npy.on_click(wrapped_func_factory(load_npy_factory('t')))
    button_load_as_x_txt.on_click(wrapped_func_factory(load_txt_factory('x')))
    button_load_as_y_txt.on_click(wrapped_func_factory(load_txt_factory('y')))
    button_load_as_quadratures_txt.on_click(wrapped_func_factory(load_txt_factory('quadratures')))
    button_x_transpose.on_click(wrapped_func_factory(transpose_factory('x')))
    button_y_transpose.on_click(wrapped_func_factory(transpose_factory('y')))


    def load_input_field():
        global xlabel
        global ylabel
        global title
        global pulseNum
        global hd_idx
        global pulse_id
        global xlim_high, xlim_low
        global ylim_high, ylim_low
        global comment
        global hist_density_max
        comment = text_comment.value
        xlabel = text_xlabel.value
        ylabel = text_ylabel.value
        title = text_title.value
        pulseNum = int_pulseNum.value
        hd_idx = int_hd_idx.value
        pulse_id = int_pulse_id.value
        hist_density_max = float_hist_density_max.value
        lims = [text_xlim_high.value, text_xlim_low.value, text_ylim_high.value, text_ylim_low.value]
        xlim_high, xlim_low, ylim_high, ylim_low = [float(i) if i !='None' else None for i in lims]

    button_input_field.on_click(lambda button: load_input_field())

    display(
        widgets.VBox([
            widgets.HBox([button_clear_output, button_init_fig, button_set_label_title, button_set_axis_lim,button_set_autoscale , button_show_fig]),
            widgets.HBox([button_add_line_y,button_add_line_xy, button_add_scatter_y, button_add_scatter_xy]),
            widgets.HBox([button_save_fig_pdf, button_save_fig_png]),
            widgets.HBox([filechooser,button_load_as_x_npy, button_load_as_y_npy]),
            widgets.HBox([button_load_as_w_npy,button_load_as_t_npy,button_load_as_x_txt,button_load_as_y_txt,button_load_as_quadratures_txt]),
            widgets.HBox([text_comment,  button_input_field]),
            widgets.HBox([text_xlabel, text_ylabel, text_title ,]),
            widgets.HBox([text_xlim_low,text_xlim_high, text_ylim_low, text_ylim_high]),
            widgets.HBox([int_pulseNum, int_hd_idx, int_pulse_id, float_hist_density_max]),
            widgets.HBox([button_x_transpose,button_y_transpose,button_calc_var, button_calc_quadratures, button_calc_moving_var]),
            widgets.HBox([button_graph_quadratures]),
            output
        ]))

In [ ]:
simple_ui()